# Few shot prompting

In [1]:
# from huggingface_hub import hf_hub_download 
import torch
from datasets import load_dataset, load_from_disk
import evaluate
import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
#from peft import LoraConfig, TaskType, get_peft_model
#from peft import PeftConfig, PeftModel
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

import pandas as pd
import numpy as np
import os

D:\cygdrive\d\projects\LLM_py38\venv\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
D:\cygdrive\d\projects\LLM_py38\venv\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
D:\cygdrive\d\projects\LLM_py38\venv\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
print(transformers.__version__)

4.35.2


In [3]:
print(torch.__version__)
print(torch.cuda.is_available())

2.3.0+cu118
True


In [4]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [8]:
llm_repo_dir = 'D:/projects/LLM'
cache_dir = '/cygdrive/d/projects/LLM/.cache'
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME'] = cache_dir + '/huggingface'
os.environ['XDG_CACHE_HOME'] = cache_dir
os.environ['HF_DATASETS_CACHE'] = cache_dir

## mistral
#model_name = "mistralai/Mistral-7B-v0.1"
#model_path = cache_dir + ''

## falcon
model_name = "tiiuae/falcon-7b-instruct"
model_path = cache_dir + '/models--tiiuae--falcon-7b-instruct/snapshots/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99'

## gpt2
#model_name = 'gpt2'
#model_path = cache_dir + '/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e'

## try gpt-neo


dataset_path = cache_dir + '/parquet/yelp_polarity' # cache_dir + '/parquet/yelp_review_full-e22176106d6e7534'
dataset_name = 'yelp_polarity' # yelp_review_full
#tokenized_data_path = cache_dir + '/tokenized_dataset_yelp_polarity_gpt2'
#tokenized_data_path = cache_dir + '/tokenized_dataset_yelp_polarity_mistral7b'
tokenized_data_path = cache_dir + '/tokenized_dataset_yelp_polarity_falcon7b'

In [9]:
if not os.path.isfile(model_path + '/tokenizer.json'):
    tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                              cache_dir=cache_dir,
                                              #torch_dtype=torch.bfloat16,
                                              device_map='auto',
                                              load_in_4bit=True) # to load tokenizer to cache
else:
    tokenizer = AutoTokenizer.from_pretrained(model_path, 
                                              cache_dir=cache_dir, 
                                              local_flies_only=True)
if not os.path.isfile(model_path + '/model.safetensors'):
    model = AutoModelForCausalLM.from_pretrained(model_name,
                                                 cache_dir=cache_dir,
                                                 device_map='auto',
                                                 load_in_4bit=True)
else:
    model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             cache_dir=cache_dir,
                                             local_files_only=True)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device) # use GPU. Do not need this if using load_in_4bit as it's already been set to the correct devices

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

D:\cygdrive\d\projects\LLM_py38\venv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\cygdrive\d\projects\LLM\.cache\models--tiiuae--falcon-7b-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

You shouldn't move a model that is dispatched using accelerate hooks.


ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [8]:
if not os.path.isdir(dataset_path):
    dataset = load_dataset(dataset_name, cache_dir=cache_dir + '/parquet')
else:
    dataset = load_dataset(dataset_path)

Found cached dataset yelp_polarity (D:/cygdrive/d/projects/LLM/.cache/parquet/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61)


  0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
dataset = load_dataset(cache_dir + '/parquet/yelp_polarity')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Using model

In [14]:
prompt = """ 
Classify the text into neutral, negative or positive.
Text: This movie is definitely one of my favorite movies of its kind. The interaction between respectable and morally strong characters is an ode to chivalry and the honor code amongst thieves and policemen.
Sentiment: positive

###

Text: This movie was actually pretty funny.
Sentiment: positive

###

Text: I hated this movie, it sucks.
Sentiment: negative

###

Text: A Perfect Spy, published in 1986, is an excellent example of the genre and an exceptionally well-written novel.
Sentiment: 
"""

In [15]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
generated_ids = model.generate(input_ids, 
                               do_sample=True, 
                               temperature=0.2, 
                               max_new_tokens=10)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
D:\cygdrive\d\projects\LLM_py38\venv\lib\site-packages\bitsandbytes\nn\modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this will be deprecated in the future in favor of the `BetterTransformer` API. Please install the latest optimum library with `pip install -U optimum` and call `model.to_bettertransformer()` to benefit from `torch.scaled_dot_product_attention` and future performance optimizations.
D:\cygdrive\d\projects\LLM_py38\venv\lib\site-packages\transformers\models\falcon\modeling_falcon.py:469: UserWarn

 
Classify the text into neutral, negative or positive.
Text: This movie is definitely one of my favorite movies of its kind. The interaction between respectable and morally strong characters is an ode to chivalry and the honor code amongst thieves and policemen.
Sentiment: positive

###

Text: This movie was actually pretty funny.
Sentiment: positive

###

Text: I hated this movie, it sucks.
Sentiment: negative

###

Text: A Perfect Spy, published in 1986, is an excellent example of the genre and an exceptionally well-written novel.
Sentiment: 

Classification:

1. Neutral:


In [16]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
generated_ids = model.generate(input_ids, 
                               do_sample=True, 
                               max_new_tokens=10,
                               top_k=10)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 
Classify the text into neutral, negative or positive.
Text: This movie is definitely one of my favorite movies of its kind. The interaction between respectable and morally strong characters is an ode to chivalry and the honor code amongst thieves and policemen.
Sentiment: positive

###

Text: This movie was actually pretty funny.
Sentiment: positive

###

Text: I hated this movie, it sucks.
Sentiment: negative

###

Text: A Perfect Spy, published in 1986, is an excellent example of the genre and an exceptionally well-written novel.
Sentiment: 
###

Text: In this book, he


In [17]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
generated_ids = model.generate(input_ids, 
                               do_sample=True, 
                               max_new_tokens=10,
                               top_p=10)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 
Classify the text into neutral, negative or positive.
Text: This movie is definitely one of my favorite movies of its kind. The interaction between respectable and morally strong characters is an ode to chivalry and the honor code amongst thieves and policemen.
Sentiment: positive

###

Text: This movie was actually pretty funny.
Sentiment: positive

###

Text: I hated this movie, it sucks.
Sentiment: negative

###

Text: A Perfect Spy, published in 1986, is an excellent example of the genre and an exceptionally well-written novel.
Sentiment: 

Classification of the text into neutral, negative


In [74]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
generated_ids = model.generate(input_ids, 
                               do_sample=True, 
                               max_new_tokens=10,
                               top_p=10,
                               top_k=0)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 
Classify the text into neutral, negative or positive.
Text: This movie is definitely one of my favorite movies of its kind. The interaction between respectable and morally strong characters is an ode to chivalry and the honor code amongst thieves and policemen.
Sentiment: positive

###

Text: This movie was actually pretty funny.
Sentiment: positive

###

Text: I hated this movie, it sucks.
Sentiment: negative

###

Text: A Perfect Spy, published in 1986, is an excellent example of the genre and an exceptionally well-written novel.
Sentiment: 

##

Text: This movie symbolically


## Using pipeline

In [ ]:
torch.manual_seed(0)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

print(next(model.parameters()).is_cuda)

In [ ]:
prompt = """Classify the text into neutral, negative or positive. 
Text: This movie is definitely one of my favorite movies of its kind. The interaction between respectable and morally strong characters is an ode to chivalry and the honor code amongst thieves and policemen.
Sentiment: positive.
Text: A Perfect Spy, published in 1986, is an excellent example of the genre and an exceptionally well-written novel.
Sentiment: 
"""

sequences = pipe(
    prompt,
    max_new_tokens=30,
    do_sample=True,
    top_k=10,
    return_full_text = False
)


## Use case: topic modeling

### Questions
- How to sample topics for prompting?
- How to do query rewriting?